In [ ]:
# Doc: https://docs.sqlalchemy.org/en/13/orm/tutorial.html
import sqlalchemy
sqlalchemy.__version__

In [ ]:
from datetime import datetime
from sqlalchemy import Column, Integer, String, Float, DateTime, Text, SmallInteger, BigInteger
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [ ]:
# Because BigInteger not auto-incremetal as primary-key in sqlite, but others
from sqlalchemy.dialects import postgresql, mysql, sqlite
BigIntegerType = BigInteger()
BigIntegerType = BigIntegerType.with_variant(postgresql.BIGINT(), 'postgresql')
BigIntegerType = BigIntegerType.with_variant(mysql.BIGINT(), 'mysql')
BigIntegerType = BigIntegerType.with_variant(sqlite.INTEGER(), 'sqlite')

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

In [ ]:
class EKUser(Base):
    __tablename__ = 'ek_users'
    id = Column(Integer, primary_key=True, autoincrement=False)  # ebayK user Id
    name = Column(String(50), nullable=False)
    address = Column(String(255), nullable=False)
    active_date = Column(DateTime, nullable=False)
    phone = Column(String(30), nullable=True)
    email = Column(String(255), nullable=True)

    def _get_url(self):
        return "https://www.ebay-kleinanzeigen.de/s-bestandsliste.html?userId={}".format(self.id)

    def __repr__(self):
        return "<EKUser(id={}, name={}, active from {})> \n  url: {}".format(self.id, self.name, self.active_date, self._get_url())

In [ ]:
class EKItem(Base):
    __tablename__ = 'ek_items'
    id = Column(BigInteger, primary_key=True, autoincrement=False)
    title = Column(String(255), nullable=False)
    price = Column(Float, nullable=False)
    release_date = Column(DateTime, nullable=False)
    stadt = Column(String(255), nullable=False)
    category = Column(String(255), nullable=False)
    sub_category = Column(String(255), nullable=False)
    status = Column(String(30), nullable=True)
    description =Column(Text, nullable=True)
    image_nb = Column(Integer, nullable=True)
    image_label = Column(String(255), nullable=True)
    seller_id = Column(Integer, ForeignKey('ek_users.id'))
    seller = relationship("EKUser", back_populates="items")

    def _get_url(self):
        return "https://www.ebay-kleinanzeigen.de/s-anzeige/{}".format(self.id)

    def __repr__(self):
        return "<EKItem(id={}, title={}, status={})> \n  url: {}".format(self.id, self.title, self.status, self._get_url())

EKUser.items = relationship("EKItem", order_by=EKItem.id, back_populates="seller")

In [ ]:
class EKViewCount(Base):
    __tablename__ = 'ek_view_counts'
    id = Column(BigIntegerType, primary_key=True)    
    count = Column(SmallInteger, nullable=False)
    at = Column(Integer, nullable=False)   # each 1 minute is 1 increment
    item_id = Column(BigInteger, ForeignKey('ek_items.id'), nullable=False)

    def __repr__(self):
        return "<EKViewCount(of item={}, count={}, at={})>".format(self.item_id, self.count, self.at)

In [ ]:
# temporary to transfer data, do not need ForeignKey
class EKMonitoringItem(Base):
    __tablename__ = 'ek_monitoring_items'
    id = Column(BigIntegerType, primary_key=True)   
    item_id = Column(BigInteger, nullable=False)
    seller_id = Column(Integer, nullable=False)
    next_count_time = Column(DateTime, nullable=False)
    count_duration = Column(Integer, nullable=False)

    def __repr__(self):
        return "<EKMonitoringItem(item_id={}, seller_id={})>".format(self.item_id, self.seller_id)

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
ek_user = EKUser(id=67461734, name='dorie', address='Home', active_date=datetime.utcnow())

In [ ]:
first_item = EKItem(id=1477722139, title="Korg damper pedal (Japan) New", price=35.0, \
    release_date=datetime.utcnow(), stadt="Munich", category="1", sub_category="2")
first_item.seller = ek_user

In [ ]:
a_count = EKViewCount(count=10, at=1, item_id=first_item.id)

In [ ]:
from datetime import timedelta
a_monitoring_item = EKMonitoringItem(item_id=1477722139, seller_id=67461734, \
    next_count_time=datetime.utcnow() + timedelta(minutes=60), count_duration=0)

In [ ]:
session.add(ek_user)
session.add(first_item)
session.add(a_count)
session.add(a_monitoring_item)
session.commit()

In [ ]:
ek_user.items

In [ ]:
first_item.seller

In [ ]:
a_count

In [ ]:
a_item = session.query(EKItem).filter_by(id=a_count.item_id).one()

In [ ]:
a_item

In [ ]:
one_monitoring_item = session.query(EKMonitoringItem).one()

In [ ]:
one_monitoring_item

In [ ]:
datetime.utcnow() - one_monitoring_item.next_count_time > timedelta(seconds=1)